In [1]:
!pip3 install python-dotenv

In [2]:
from dotenv import load_dotenv # add this line
load_dotenv() # add this line

accesstoken = os.getenv('ACCESS_TOKEN')
accesstokensecret = os.getenv('ACCESS_TOKEN_SECRET')
apikey = os.getenv('API_KEY')
apisecretkey = os.getenv('API_SECRET_KEY')
print(accesstoken)

1027542862861426688-IbwHGihEBcGrCeaz9P5gZ0iczYlWwj


In [3]:
import pandas as pd
import numpy as np
import tweepy as tw

In [4]:
# Set up the auth tokens
auth = tw.OAuthHandler(apikey,apisecretkey)
auth.set_access_token(accesstoken, accesstokensecret)

api = tw.API(auth,wait_on_rate_limit = True)


In [5]:
search_words = '#coronavirus'
date_since = '2020-3-10'

In [7]:
tweets = tw.Cursor(api.search,q=search_words,since=date_since,lang="en").items(100)

In [8]:
tweets

In [9]:
tweet_details = [[tweet.geo,tweet.text,tweet.user.screen_name,tweet.user.location] for tweet in tweets]

In [10]:
tweet_df = pd.DataFrame(data=tweet_details,columns=['geo','text','user',"location"])

In [ ]:
pd.set_option('max_colwidth', 800)
tweet_df.head(20)

,geo,text,user,location
0,None,"RT @EmenDior: Due to #COVID19 aka #Coronavirus Ill be sending the first 900 people to like &amp; retweet this $300 -$1500 through CashApp, Appl…",caseyjohnson90,"Manchester, England"
1,None,Fed Chair Jerome Powell said that US economic outlook was “extraordinarily uncertain” and depended on containing… https://t.co/jQqIREJeEn,acemaxx,Zurich
2,None,RT @cedrec_news: We're collating relevant H&amp;S coronavirus guidance through our COVID-19 advice In Focus - a central hub for professionals.…,SfLtd,United Kingdom
3,None,RT @EBRD: How do #cities recover sustainably in the face of the #coronavirus pandemic? \n\nJoin our discussion at #LCAW2020 with our Josué Ta…,ODIclimate,
4,None,"RT @RealJamesWoods: Rioters and looters and mobs packed together like sardines, howling and spewing spittle, have swarmed over the urban la…",JujuB312,United States
5,None,"RT @KeithSinclair5: Hey Scottish unionists, here’s global public health expert Prof Devi Sridhar telling C4 News “what Scotland got right”…",sittingbourneTA,"Sittingbourne, Kent"
6,None,@ANI F**k #ChinaVirus ! All world is in trouble today by them 🇨🇳 ! #coronavirus Now cresting more new viruses 🦠 #NewVirus #PigVirus,raghav2love,Chandigarh
7,None,"The #coronavirus pandemic has led a growing number of Westerners to see #China as a top power, with the lead of the… https://t.co/EDByX4eog3",DeccanHerald,India
8,None,RT @UNGeneva: 𝑻𝒉𝒊𝒔 𝒊𝒔 𝒇𝒂𝒓 𝒇𝒓𝒐𝒎 𝒐𝒗𝒆𝒓. \n\n𝑻𝒉𝒆 𝒑𝒂𝒏𝒅𝒆𝒎𝒊𝒄 𝒊𝒔 𝒔𝒑𝒆𝒆𝒅𝒊𝒏𝒈 𝒖𝒑.\n\n𝑾𝒆 𝒂𝒓𝒆 𝒂𝒍𝒍 𝒊𝒏 𝒕𝒉𝒊𝒔 𝒕𝒐𝒈𝒆𝒕𝒉𝒆𝒓 𝒂𝒏𝒅 𝒇𝒐𝒓 𝒂 𝒍𝒐𝒏𝒈 𝒉𝒂𝒖𝒍.\n\n-- @DrTedros \n#COVID…,RD_Omboto,"Nairobi, Kenya"
9,None,RT @PCD_UK: PCD and Covid19 Survey roundup 29/06/2020:\n\nThe data collected is invaluable to assess the severity of #coronavirus on PCDers.…,JaneSLucas1,Salisbury and Southampton UK


In [ ]:
tweet_df.user.value_counts()

line_ryder         2
mynation_F4JSTC    2
DeccanHerald       2
ODIclimate         1
jaigoenka          1
                  ..
Elljay1011         1
MuratEk58506559    1
jannice            1
Saadia37765706     1
climatebabes       1
Name: user, Length: 97, dtype: int64

In [ ]:
tweet_df.location.value_counts()

                           24
India                       6
London, England             3
England, United Kingdom     2
London                      2
                           ..
Nottinghamshire             1
USA                         1
Edinburgh, Scotland         1
Teddington, England         1
Nairobi, Kenya              1
Name: location, Length: 68, dtype: int64

In [ ]:
import re
def clean_tweets(text):
  text = re.sub("RT @[\w]*:", "", text)
  text = re.sub("@[\w]*", "", text)
  text = re.sub("https?://[A-Za-z0-9./]*", "", text)
  text = re.sub("\n", "", text)
  return text

In [ ]:
tweet_df['text'] = tweet_df['text'].apply(lambda x: clean_tweets(x))

In [ ]:
tweet_df.head(20)

,geo,text,user,location
0,None,"Due to #COVID19 aka #Coronavirus Ill be sending the first 900 people to like &amp; retweet this $300 -$1500 through CashApp, Appl…",caseyjohnson90,"Manchester, England"
1,None,Fed Chair Jerome Powell said that US economic outlook was “extraordinarily uncertain” and depended on containing…,acemaxx,Zurich
2,None,We're collating relevant H&amp;S coronavirus guidance through our COVID-19 advice In Focus - a central hub for professionals.…,SfLtd,United Kingdom
3,None,How do #cities recover sustainably in the face of the #coronavirus pandemic? Join our discussion at #LCAW2020 with our Josué Ta…,ODIclimate,
4,None,"Rioters and looters and mobs packed together like sardines, howling and spewing spittle, have swarmed over the urban la…",JujuB312,United States
5,None,"Hey Scottish unionists, here’s global public health expert Prof Devi Sridhar telling C4 News “what Scotland got right”…",sittingbourneTA,"Sittingbourne, Kent"
6,None,F**k #ChinaVirus ! All world is in trouble today by them 🇨🇳 ! #coronavirus Now cresting more new viruses 🦠 #NewVirus #PigVirus,raghav2love,Chandigarh
7,None,"The #coronavirus pandemic has led a growing number of Westerners to see #China as a top power, with the lead of the…",DeccanHerald,India
8,None,𝑻𝒉𝒊𝒔 𝒊𝒔 𝒇𝒂𝒓 𝒇𝒓𝒐𝒎 𝒐𝒗𝒆𝒓. 𝑻𝒉𝒆 𝒑𝒂𝒏𝒅𝒆𝒎𝒊𝒄 𝒊𝒔 𝒔𝒑𝒆𝒆𝒅𝒊𝒏𝒈 𝒖𝒑.𝑾𝒆 𝒂𝒓𝒆 𝒂𝒍𝒍 𝒊𝒏 𝒕𝒉𝒊𝒔 𝒕𝒐𝒈𝒆𝒕𝒉𝒆𝒓 𝒂𝒏𝒅 𝒇𝒐𝒓 𝒂 𝒍𝒐𝒏𝒈 𝒉𝒂𝒖𝒍.-- #COVID…,RD_Omboto,"Nairobi, Kenya"
9,None,PCD and Covid19 Survey roundup 29/06/2020:The data collected is invaluable to assess the severity of #coronavirus on PCDers.…,JaneSLucas1,Salisbury and Southampton UK


In [ ]:
tweet_df.to_csv('tweets.csv')

In [ ]:
!ls

drive  sample_data  tweets.csv


In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [ ]:
tweet_df['text'].apply(lambda x: [print("\tText : {}, Entity : {}".format(ent.text,ent.label_)) if (not ent.text.startswith('#')) else "" for ent in nlp(x).ents])

	Text : first, Entity : ORDINAL
	Text : 300, Entity : MONEY
	Text : CashApp, Entity : PRODUCT
	Text : Appl, Entity : PERSON
	Text : Fed, Entity : ORG
	Text : Jerome Powell, Entity : PERSON
	Text : US, Entity : GPE
	Text : LCAW2020, Entity : MONEY
	Text : Josué Ta, Entity : PERSON
	Text : Scottish, Entity : NORP
	Text : Prof Devi Sridhar, Entity : PERSON
	Text : C4 News, Entity : ORG
	Text : Scotland, Entity : GPE
	Text : ChinaVirus, Entity : PERSON
	Text : today, Entity : DATE
	Text : 🇳, Entity : ORG
	Text : coronavirus, Entity : PERSON
	Text : China, Entity : GPE
	Text : COVID, Entity : PERSON
	Text : PCD, Entity : ORG
	Text : 29/06/2020, Entity : CARDINAL
	Text : China, Entity : GPE
	Text : 14 June 2020, Entity : DATE
	Text : ANC, Entity : ORG
	Text : Arizona, Entity : PERSON
	Text : 30 days, Entity : DATE
	Text : 86th Entry, Entity : ORG
	Text : Coronavirus Poetry, Entity : ORG
	Text : 10, Entity : CARDINAL
	Text : COVID19, Entity : PERSON
	Text : the World Health Organisation', Ent

0     [None, None, None, None]
1           [None, None, None]
2                           []
3             [, , None, None]
4                           []
                ...           
95          [None, None, None]
96                    [None, ]
97                [None, None]
98                          []
99                          []
Name: text, Length: 100, dtype: object

In [ ]:
tweet_df['entities'] = tweet_df['text'].apply(lambda x: [(ent.text, ent.label_) if (not ent.text.startswith('#')) else "" for ent in nlp(x).ents])

In [ ]:
tweet_df.head(20)

,geo,text,user,location,entities
0,None,"Due to #COVID19 aka #Coronavirus Ill be sending the first 900 people to like &amp; retweet this $300 -$1500 through CashApp, Appl…",caseyjohnson90,"Manchester, England","[(first, ORDINAL), (300, MONEY), (CashApp, PRODUCT), (Appl, PERSON)]"
1,None,Fed Chair Jerome Powell said that US economic outlook was “extraordinarily uncertain” and depended on containing…,acemaxx,Zurich,"[(Fed, ORG), (Jerome Powell, PERSON), (US, GPE)]"
2,None,We're collating relevant H&amp;S coronavirus guidance through our COVID-19 advice In Focus - a central hub for professionals.…,SfLtd,United Kingdom,[]
3,None,How do #cities recover sustainably in the face of the #coronavirus pandemic? Join our discussion at #LCAW2020 with our Josué Ta…,ODIclimate,,"[, , (LCAW2020, MONEY), (Josué Ta, PERSON)]"
4,None,"Rioters and looters and mobs packed together like sardines, howling and spewing spittle, have swarmed over the urban la…",JujuB312,United States,[]
5,None,"Hey Scottish unionists, here’s global public health expert Prof Devi Sridhar telling C4 News “what Scotland got right”…",sittingbourneTA,"Sittingbourne, Kent","[(Scottish, NORP), (Prof Devi Sridhar, PERSON), (C4 News, ORG), (Scotland, GPE)]"
6,None,F**k #ChinaVirus ! All world is in trouble today by them 🇨🇳 ! #coronavirus Now cresting more new viruses 🦠 #NewVirus #PigVirus,raghav2love,Chandigarh,"[(ChinaVirus, PERSON), (today, DATE), (🇳, ORG), (coronavirus, PERSON)]"
7,None,"The #coronavirus pandemic has led a growing number of Westerners to see #China as a top power, with the lead of the…",DeccanHerald,India,"[, (China, GPE)]"
8,None,𝑻𝒉𝒊𝒔 𝒊𝒔 𝒇𝒂𝒓 𝒇𝒓𝒐𝒎 𝒐𝒗𝒆𝒓. 𝑻𝒉𝒆 𝒑𝒂𝒏𝒅𝒆𝒎𝒊𝒄 𝒊𝒔 𝒔𝒑𝒆𝒆𝒅𝒊𝒏𝒈 𝒖𝒑.𝑾𝒆 𝒂𝒓𝒆 𝒂𝒍𝒍 𝒊𝒏 𝒕𝒉𝒊𝒔 𝒕𝒐𝒈𝒆𝒕𝒉𝒆𝒓 𝒂𝒏𝒅 𝒇𝒐𝒓 𝒂 𝒍𝒐𝒏𝒈 𝒉𝒂𝒖𝒍.-- #COVID…,RD_Omboto,"Nairobi, Kenya","[(COVID, PERSON)]"
9,None,PCD and Covid19 Survey roundup 29/06/2020:The data collected is invaluable to assess the severity of #coronavirus on PCDers.…,JaneSLucas1,Salisbury and Southampton UK,"[(PCD, ORG), (29/06/2020, CARDINAL)]"


In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
sid = SentimentIntensityAnalyzer()

In [ ]:
tweet_df['sentiment'] = tweet_df['text'].apply(lambda x: sid.polarity_scores(x))

In [ ]:
tweet_df.head(20)

,geo,text,user,location,entities,sentiment
0,None,"Due to #COVID19 aka #Coronavirus Ill be sending the first 900 people to like &amp; retweet this $300 -$1500 through CashApp, Appl…",caseyjohnson90,"Manchester, England","[(first, ORDINAL), (300, MONEY), (CashApp, PRODUCT), (Appl, PERSON)]","{'neg': 0.111, 'neu': 0.791, 'pos': 0.099, 'compound': -0.0772}"
1,None,Fed Chair Jerome Powell said that US economic outlook was “extraordinarily uncertain” and depended on containing…,acemaxx,Zurich,"[(Fed, ORG), (Jerome Powell, PERSON), (US, GPE)]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}"
2,None,We're collating relevant H&amp;S coronavirus guidance through our COVID-19 advice In Focus - a central hub for professionals.…,SfLtd,United Kingdom,[],"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}"
3,None,How do #cities recover sustainably in the face of the #coronavirus pandemic? Join our discussion at #LCAW2020 with our Josué Ta…,ODIclimate,,"[, , (LCAW2020, MONEY), (Josué Ta, PERSON)]","{'neg': 0.0, 'neu': 0.901, 'pos': 0.099, 'compound': 0.296}"
4,None,"Rioters and looters and mobs packed together like sardines, howling and spewing spittle, have swarmed over the urban la…",JujuB312,United States,[],"{'neg': 0.0, 'neu': 0.878, 'pos': 0.122, 'compound': 0.3612}"
5,None,"Hey Scottish unionists, here’s global public health expert Prof Devi Sridhar telling C4 News “what Scotland got right”…",sittingbourneTA,"Sittingbourne, Kent","[(Scottish, NORP), (Prof Devi Sridhar, PERSON), (C4 News, ORG), (Scotland, GPE)]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}"
6,None,F**k #ChinaVirus ! All world is in trouble today by them 🇨🇳 ! #coronavirus Now cresting more new viruses 🦠 #NewVirus #PigVirus,raghav2love,Chandigarh,"[(ChinaVirus, PERSON), (today, DATE), (🇳, ORG), (coronavirus, PERSON)]","{'neg': 0.154, 'neu': 0.846, 'pos': 0.0, 'compound': -0.508}"
7,None,"The #coronavirus pandemic has led a growing number of Westerners to see #China as a top power, with the lead of the…",DeccanHerald,India,"[, (China, GPE)]","{'neg': 0.0, 'neu': 0.78, 'pos': 0.22, 'compound': 0.4215}"
8,None,𝑻𝒉𝒊𝒔 𝒊𝒔 𝒇𝒂𝒓 𝒇𝒓𝒐𝒎 𝒐𝒗𝒆𝒓. 𝑻𝒉𝒆 𝒑𝒂𝒏𝒅𝒆𝒎𝒊𝒄 𝒊𝒔 𝒔𝒑𝒆𝒆𝒅𝒊𝒏𝒈 𝒖𝒑.𝑾𝒆 𝒂𝒓𝒆 𝒂𝒍𝒍 𝒊𝒏 𝒕𝒉𝒊𝒔 𝒕𝒐𝒈𝒆𝒕𝒉𝒆𝒓 𝒂𝒏𝒅 𝒇𝒐𝒓 𝒂 𝒍𝒐𝒏𝒈 𝒉𝒂𝒖𝒍.-- #COVID…,RD_Omboto,"Nairobi, Kenya","[(COVID, PERSON)]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}"
9,None,PCD and Covid19 Survey roundup 29/06/2020:The data collected is invaluable to assess the severity of #coronavirus on PCDers.…,JaneSLucas1,Salisbury and Southampton UK,"[(PCD, ORG), (29/06/2020, CARDINAL)]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}"


In [ ]:
import os

In [ ]:
import geopy

In [ ]:
dir (geopy)

['ArcGIS',
 'AzureMaps',
 'Baidu',
 'Bing',
 'DataBC',
 'GeoNames',
 'GeocodeEarth',
 'GeocodeFarm',
 'GoogleV3',
 'Here',
 'IGNFrance',
 'LiveAddress',
 'Location',
 'MapBox',
 'Nominatim',
 'OpenCage',
 'OpenMapQuest',
 'Pelias',
 'Photon',
 'PickPoint',
 'Point',
 'TomTom',
 'What3Words',
 'Yandex',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'compat',
 'exc',
 'format',
 'geocoders',
 'get_geocoder_for_service',
 'location',
 'point',
 'units',
 'util']

In [ ]:
from geopy.geocoders import Nominatim

In [ ]:
nom = Nominatim()

/usr/local/lib/python3.6/dist-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


In [ ]:
tweet_df["coordinates"]= tweet_df["location"].apply(nom.geocode)

In [ ]:
tweet_df.head(20)

,geo,text,user,location,entities,sentiment,coordinates
0,None,"Due to #COVID19 aka #Coronavirus Ill be sending the first 900 people to like &amp; retweet this $300 -$1500 through CashApp, Appl…",caseyjohnson90,"Manchester, England","[(first, ORDINAL), (300, MONEY), (CashApp, PRODUCT), (Appl, PERSON)]","{'neg': 0.111, 'neu': 0.791, 'pos': 0.099, 'compound': -0.0772}","(Manchester, Greater Manchester, North West England, England, United Kingdom, (53.4794892, -2.2451148))"
1,None,Fed Chair Jerome Powell said that US economic outlook was “extraordinarily uncertain” and depended on containing…,acemaxx,Zurich,"[(Fed, ORG), (Jerome Powell, PERSON), (US, GPE)]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}","(Zürich, Bezirk Zürich, Zürich, Switzerland, (47.3723941, 8.5423328))"
2,None,We're collating relevant H&amp;S coronavirus guidance through our COVID-19 advice In Focus - a central hub for professionals.…,SfLtd,United Kingdom,[],"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}","(United Kingdom, (54.7023545, -3.2765753))"
3,None,How do #cities recover sustainably in the face of the #coronavirus pandemic? Join our discussion at #LCAW2020 with our Josué Ta…,ODIclimate,,"[, , (LCAW2020, MONEY), (Josué Ta, PERSON)]","{'neg': 0.0, 'neu': 0.901, 'pos': 0.099, 'compound': 0.296}",None
4,None,"Rioters and looters and mobs packed together like sardines, howling and spewing spittle, have swarmed over the urban la…",JujuB312,United States,[],"{'neg': 0.0, 'neu': 0.878, 'pos': 0.122, 'compound': 0.3612}","(United States, (39.7837304, -100.4458825))"
5,None,"Hey Scottish unionists, here’s global public health expert Prof Devi Sridhar telling C4 News “what Scotland got right”…",sittingbourneTA,"Sittingbourne, Kent","[(Scottish, NORP), (Prof Devi Sridhar, PERSON), (C4 News, ORG), (Scotland, GPE)]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}","(Sittingbourne, Kent, South East, England, ME10 4AL, United Kingdom, (51.3397366, 0.7342319))"
6,None,F**k #ChinaVirus ! All world is in trouble today by them 🇨🇳 ! #coronavirus Now cresting more new viruses 🦠 #NewVirus #PigVirus,raghav2love,Chandigarh,"[(ChinaVirus, PERSON), (today, DATE), (🇳, ORG), (coronavirus, PERSON)]","{'neg': 0.154, 'neu': 0.846, 'pos': 0.0, 'compound': -0.508}","(Chandigarh, India, (30.7194022, 76.7646552))"
7,None,"The #coronavirus pandemic has led a growing number of Westerners to see #China as a top power, with the lead of the…",DeccanHerald,India,"[, (China, GPE)]","{'neg': 0.0, 'neu': 0.78, 'pos': 0.22, 'compound': 0.4215}","(India, (22.3511148, 78.6677428))"
8,None,𝑻𝒉𝒊𝒔 𝒊𝒔 𝒇𝒂𝒓 𝒇𝒓𝒐𝒎 𝒐𝒗𝒆𝒓. 𝑻𝒉𝒆 𝒑𝒂𝒏𝒅𝒆𝒎𝒊𝒄 𝒊𝒔 𝒔𝒑𝒆𝒆𝒅𝒊𝒏𝒈 𝒖𝒑.𝑾𝒆 𝒂𝒓𝒆 𝒂𝒍𝒍 𝒊𝒏 𝒕𝒉𝒊𝒔 𝒕𝒐𝒈𝒆𝒕𝒉𝒆𝒓 𝒂𝒏𝒅 𝒇𝒐𝒓 𝒂 𝒍𝒐𝒏𝒈 𝒉𝒂𝒖𝒍.-- #COVID…,RD_Omboto,"Nairobi, Kenya","[(COVID, PERSON)]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}","(Nairobi, Kenya, (-1.2832533, 36.8172449))"
9,None,PCD and Covid19 Survey roundup 29/06/2020:The data collected is invaluable to assess the severity of #coronavirus on PCDers.…,JaneSLucas1,Salisbury and Southampton UK,"[(PCD, ORG), (29/06/2020, CARDINAL)]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}","(Salisbury Street, The Polygon, Southampton, South East, England, SO15 2AE, United Kingdom, (50.9113743, -1.4050786))"


In [ ]:
tweet_df['latitude'] = tweet_df['coordinates'].apply(lambda x: x.latitude if x != None else None)

In [ ]:
tweet_df['longitude'] = tweet_df['coordinates'].apply(lambda x: x.longitude if x != None else None)

In [ ]:
tweet_df.head(20)

,geo,text,user,location,entities,sentiment,coordinates,latitude,longitude
0,None,"Due to #COVID19 aka #Coronavirus Ill be sending the first 900 people to like &amp; retweet this $300 -$1500 through CashApp, Appl…",caseyjohnson90,"Manchester, England","[(first, ORDINAL), (300, MONEY), (CashApp, PRODUCT), (Appl, PERSON)]","{'neg': 0.111, 'neu': 0.791, 'pos': 0.099, 'compound': -0.0772}","(Manchester, Greater Manchester, North West England, England, United Kingdom, (53.4794892, -2.2451148))",53.479489,-2.245115
1,None,Fed Chair Jerome Powell said that US economic outlook was “extraordinarily uncertain” and depended on containing…,acemaxx,Zurich,"[(Fed, ORG), (Jerome Powell, PERSON), (US, GPE)]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}","(Zürich, Bezirk Zürich, Zürich, Switzerland, (47.3723941, 8.5423328))",47.372394,8.542333
2,None,We're collating relevant H&amp;S coronavirus guidance through our COVID-19 advice In Focus - a central hub for professionals.…,SfLtd,United Kingdom,[],"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}","(United Kingdom, (54.7023545, -3.2765753))",54.702354,-3.276575
3,None,How do #cities recover sustainably in the face of the #coronavirus pandemic? Join our discussion at #LCAW2020 with our Josué Ta…,ODIclimate,,"[, , (LCAW2020, MONEY), (Josué Ta, PERSON)]","{'neg': 0.0, 'neu': 0.901, 'pos': 0.099, 'compound': 0.296}",None,NaN,NaN
4,None,"Rioters and looters and mobs packed together like sardines, howling and spewing spittle, have swarmed over the urban la…",JujuB312,United States,[],"{'neg': 0.0, 'neu': 0.878, 'pos': 0.122, 'compound': 0.3612}","(United States, (39.7837304, -100.4458825))",39.783730,-100.445882
5,None,"Hey Scottish unionists, here’s global public health expert Prof Devi Sridhar telling C4 News “what Scotland got right”…",sittingbourneTA,"Sittingbourne, Kent","[(Scottish, NORP), (Prof Devi Sridhar, PERSON), (C4 News, ORG), (Scotland, GPE)]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}","(Sittingbourne, Kent, South East, England, ME10 4AL, United Kingdom, (51.3397366, 0.7342319))",51.339737,0.734232
6,None,F**k #ChinaVirus ! All world is in trouble today by them 🇨🇳 ! #coronavirus Now cresting more new viruses 🦠 #NewVirus #PigVirus,raghav2love,Chandigarh,"[(ChinaVirus, PERSON), (today, DATE), (🇳, ORG), (coronavirus, PERSON)]","{'neg': 0.154, 'neu': 0.846, 'pos': 0.0, 'compound': -0.508}","(Chandigarh, India, (30.7194022, 76.7646552))",30.719402,76.764655
7,None,"The #coronavirus pandemic has led a growing number of Westerners to see #China as a top power, with the lead of the…",DeccanHerald,India,"[, (China, GPE)]","{'neg': 0.0, 'neu': 0.78, 'pos': 0.22, 'compound': 0.4215}","(India, (22.3511148, 78.6677428))",22.351115,78.667743
8,None,𝑻𝒉𝒊𝒔 𝒊𝒔 𝒇𝒂𝒓 𝒇𝒓𝒐𝒎 𝒐𝒗𝒆𝒓. 𝑻𝒉𝒆 𝒑𝒂𝒏𝒅𝒆𝒎𝒊𝒄 𝒊𝒔 𝒔𝒑𝒆𝒆𝒅𝒊𝒏𝒈 𝒖𝒑.𝑾𝒆 𝒂𝒓𝒆 𝒂𝒍𝒍 𝒊𝒏 𝒕𝒉𝒊𝒔 𝒕𝒐𝒈𝒆𝒕𝒉𝒆𝒓 𝒂𝒏𝒅 𝒇𝒐𝒓 𝒂 𝒍𝒐𝒏𝒈 𝒉𝒂𝒖𝒍.-- #COVID…,RD_Omboto,"Nairobi, Kenya","[(COVID, PERSON)]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}","(Nairobi, Kenya, (-1.2832533, 36.8172449))",-1.283253,36.817245
9,None,PCD and Covid19 Survey roundup 29/06/2020:The data collected is invaluable to assess the severity of #coronavirus on PCDers.…,JaneSLucas1,Salisbury and Southampton UK,"[(PCD, ORG), (29/06/2020, CARDINAL)]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}","(Salisbury Street, The Polygon, Southampton, South East, England, SO15 2AE, United Kingdom, (50.9113743, -1.4050786))",50.911374,-1.405079
